# Similar Hot Words

In [ ]:
from transformers import file_utils
import spacy
import pandas as pd
from InstructorEmbedding import INSTRUCTOR
from sklearn.metrics.pairwise import cosine_similarity
file_utils.CURL_TIMEOUT = 400  # Increase timeout as 'hkunlp/instructor-large' takes longer to load
nlp = spacy.load("en_core_web_sm")
from tqdm import tqdm

c:\VirtualEnv\asr\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


## Load Data and Model

In [2]:
data = pd.read_csv('../data/cv-valid-dev.csv')
data.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN


In [3]:
model = INSTRUCTOR('hkunlp/instructor-large')

load INSTRUCTOR_Transformer
max_seq_length  512


c:\VirtualEnv\asr\Lib\site-packages\sentence_transformers\models\Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join(inp

## Compute Similarity between Phrases and Hot Words

Covert Hot Words to Embeddings using Text Embedding Model

In [4]:
hotwords = ['be careful', 'destroy', 'stranger']
hotwords_embeddings = {}

for i in hotwords:
    hotwords_embeddings[i] = model.encode(i).reshape(1,-1)

Definie similarity function that takes in a text and split it into phrases. Each phrase is then converted into embeddings to compute cosine similarity with each of the hot words. Should there be a cosine similarity greater than 0.9, the result will return True, otherwise False.

In [13]:
def similarity(text):
    sent = nlp(text)
    phrases = [phrase.text for phrase in sent.noun_chunks]
    if phrases == []:
        phrases = [text]
    result = False
    for i in phrases:
        for j in range(len(hotwords)):
            if cosine_similarity(model.encode(i).reshape(1,-1), hotwords_embeddings[hotwords[j]]) >= 0.9:                
                result = True
                break
        if result == True:
            break
    return result

In [19]:
for i in tqdm(range(data.shape[0])):
    data.loc[i,'similarity'] = similarity(data.loc[i,'text'])
    if i % 100 == 0:
        data.to_csv('cv-valid-dev-similarity.csv', index = False)

100%|██████████| 4076/4076 [2:57:11<00:00,  2.61s/it]   


## Save File to CSV

In [20]:
data.to_csv('cv-valid-dev-similarity.csv', index = False)